# Let the Syntax guide You

<!-- TODO: fix the syntax inconsistencies for Exp1 - currently stmt_list has a nullable prefix -->


<!--
\index{syntax directed language processing}
-->

Now that we have an understanding of what parsing entails we can build our first interpreters and simple translators.
For a certain class of languages we can do our processing as soon as we recognize syntactic structures, that means
we can do our processing right in the embedded actions of a grammar.
This is called *syntax directed language processing*.
We start this chapter by looking at a syntax directed interpreter for Exp1 based on a hand-coded recursive descent parser.
We then implement the same interpreter by replacing the hand-coded parser with a machine generated LR(1) parser taking advantage of the embedded actions Ply provides.
The last example we look at is a pretty printer for Exp1.  Pretty printers are examples of simple translators.  Here we implement the pretty printer in a syntax directed manner based on the embedded rules in the parser generated by Ply.

In [1]:
# let the notebook access the code folder
import sys
sys.path.insert(1,"code")

## What is Syntax-Directed Interpretation?

<!--
\index{interpreter}
\index{interpretation}
\index{interpretation!algebraic terms}
\index{interpretation!assignment statement}
\index{interpretation!syntax-directed}
\index{syntax-directed interpretation}
-->

According to our classification of language processors in Chapter 1 an interpreter reads a program and executes the program
directly (see Chapter 1 Figure 6).
We accomplish this by interpreting the syntactic structures as soon as we parse them.
This is called *syntax-directed interpretation* where we execute the semantic rules of the language as soon as we recognize
the corresponding syntactic structures.

What exactly do we mean by interpretation?
In order to get a better idea of what interpretation is we turn to a language that you are very familiar with: algebra.
Consider the algebraic expression,
```
x = 3
```
We interpret this expression by first interpreting the symbol `3` as the mathematical value three, we then interpret the symbol
`x` as a variable, and because the variable appears to the left of the symbol `=`  we assign the value three to the variable `x`.
Now consider the term,
```
y = 2 + x
```
In order to interpret this term we first figure out what value is assigned to the variable `x`, we then interpret the symbol `2` as the mathematical
value two, and finally we compute the value of the right term by interpreting the `+` symbol as addition computing
the value five (if we assume that `x` has the value three from the previous example).  In order to complete the interpretation of this algebraic term we again interpret
the `=` as the assignment of the value five to the variable `y`.

<!--
\index{interpretation!syntax directed}
\index{syntax-direct interpretation}
-->

One thing you probably noticed at this point is that the interpretation of algebraic terms is *bottom-up*, that is, it starts with the operands
that are immediately computable, such as constant symbols or variables,  and works its way up to the top-level operator which in this case is the assignment operator.

>This approach to interpretation is called syntax-directed interpretation because the interpretation is guided by the syntactic structure of the terms.

We often think of this interpretation as semantics because the interpretation provides a behavioral view of the term or program.

## An Interpreter for Exp1 using a Recursive Descent Parser

### The Interpretation of Exp1 Programs

Now recall the syntax of our Exp1 language:
```
prog : stmt_list 

stmt_list : stmt_list stmt
          | empty

stmt : PRINT exp ';'
     | STORE var exp ';'

exp : '+' exp exp
    | '-' exp exp
    | '(' exp ')'
    | var
    | num
	
var : NAME 
num : NUMBER
```
It is the language of pre-fix expressions and has two statements.  One to print values of expressions to the terminal and the other to store the value of an expression in a variable.
In order to see what syntax-directed interpretation looks like for our Exp1 language let us start with the parse tree for the program,
```
store y + 2 x ;
```
Figure 1 shows the parse tree for this program.  It is clear from the structure of the tree that in order to compute a value to store into variable `y` we would have to interpret the tree starting at the right side leaves and then keep interpreting the operators and computing the values along the tree branches in the direction of the red arrows.  One way to visualize syntax directed interpretation is that values percolate from the tree leaves up to the root. In our case, once interpretation reaches the root of the parse tree the value computed thus far is stored in the variable `y`.

***
<center>
<img src="figures/chap03/1/figure/Slide1.jpg" alt="">
Fig 1. Interpreting the parse tree for the program `store y + 2 x ;`.
</center>

***

### Syntax Directed Interpretation of Expressions

Now it turns out that we can achieve the same interpretation behavior that we showed above in a parser without having to construct an explicit parse
tree.
Consider the non-terminal `exp` defined in
the Exp1 grammar as,
```
exp : '+' exp exp
    | '-' exp exp
    | '(' exp ')'
    | var
    | num

```
Here we look at a hand-built recursive descent parser for this non-terminal.
In order to enable syntax directed interpretation of expressions all we have to do is 
allow return values from the parsing functions.

The recursive descent parsing function for expressions,

In [2]:
# %load -s exp code/exp1_rinterp
def exp():
    tok = token_stream.pointer()
    
    if tok.type == '+':
        token_stream.next() # match '+'
        return exp() + exp()
    
    elif tok.type == '-':
        token_stream.next() # match '-'
        return exp() - exp()
    
    elif tok.type == '(':
        token_stream.next() # match '('
        val = exp()
        token_stream.next() # match ')'
        return val
    
    elif tok.type == 'NAME':
        return var()
    
    elif tok.type == 'NUMBER':
        return num()
    
    else:
        raise SyntaxError('unexpected symbol {} while parsing'.format(tok.value))


An interesting observation is that the function call sequence representing the non-terminals of the grammar implicitly build a parse tree.  Consider parsing the sub-expression `+ 1 2` that appears in the parse tree in Figure 1 with our
`exp()` funtion.  It not difficult to see that the pattern of function calls during parsing match exactly the non-terminals in
the parse tree (try tracing the function calls for that expression!).  Furthermore, the return values of the parsing 
function percolate the values from the leaves of the implicit parse tree up in such a way that the function call to `exp()` in order to parse `+ 1 2` will return the value three.

A closer look at the parsing function `exp()` itself.  For the tokens `+` and `-` we see two recursive function calls itself and then given the respective returned values the appropriate arithmetic operation in order to compute the return value, that is, at this point we take the two values that propagated up from the subexpressions, add or subtract them as appropriate, and return the newly computed value. 

Something very similar happens with the token `(`.  Here we simply return the value of the parenthesized expression.

When we encounter variables and numbers in the expression the appropriate parsing functions are called.
It turns out that both of those function represent the termination cases for the recursion and provide values for the leaves.  The function `num()` simply returns the value of the number encountered and the function `var()` returns the value associated with the variable name.

In short, the function `exp()` represents a recursive function that will recurse while parsing expressions until it
finds either a variable to parse with `var()` or a number to parse with `num()`.
At that point recursion stops and starts to unwind percolating values up the implicit parse tree.
In this way we see computed values percolating up from the leaves to the root of the tree where they can then be used.

### The Syntax Directed Interpretation of Variables and Numbers

Here are the parsing functions for variables and numbers.  Notice that the `var()` function looks up the value associated with the variable name in a symbol table.   We made a design choice that if the variable has not been previously initialized we return the value zero.

In [3]:
# %load -s var code/exp1_rinterp
def var():
    tok = token_stream.pointer()
    
    if tok.type == 'NAME':
        token_stream.next()
        return symbol_table.get(tok.value, 0) # return 0 if not found
    
    else:
        raise SyntaxError('unexpected symbol {} while parsing'.format(tok.value))


In [4]:
# %load -s num code/exp1_rinterp
def num():
    tok = token_stream.pointer()
    
    if tok.type == 'NUMBER':
        token_stream.next()
        return tok.value
    
    else:
        raise SyntaxError('unexpected symbol {} while parsing'.format(tok.value))


### Interpreting an Expression

In order to run those functions we need set up our lexical analysis, token stream, and symbol table.    For our lexical analysis we use the lexer for Exp1 from Chapter 2, `exp1_lex.lexer`.  The class `TokenStream` converts a character stream into a token stream using the given lexical analyzer.  Our symbol table is a dictionary in order to associate names with values.

In [5]:
from exp1_lex import lexer
from grammar_stuff import TokenStream

In [6]:
symbol_table = dict()

In [7]:
input_stream = "+ 1 x0"

In [8]:
token_stream = TokenStream(lexer, input_stream)

Here is a little program that illustrates how our token stream works.  The program reads the tokens from the token stream one by one and prints out the token name together with the token value.

In [9]:
while not token_stream.end_of_file():
    tok = token_stream.pointer()
    print("Token: name={} value={}".format(tok.type, tok.value))
    token_stream.next()

Token: name=+ value=+
Token: name=NUMBER value=1
Token: name=NAME value=x0


Our token stream works nicely. Now, let's put this to use for parsing and evaluating Exp1 expressions using our recursive descent parser.

In [10]:
input_stream = "+ 1 2"

In [11]:
token_stream = TokenStream(lexer, input_stream)

In [12]:
print(exp())

3


Yes! Given an input stream `"+ 1 2"` a call to the  `exp()` parsing function returns the value three, as we would expect from our previous discussion.

Let's try this on something a bit more complicated,

In [13]:
input_stream = "(- (+ 1 2) 1)"

In [14]:
token_stream = TokenStream(lexer, input_stream)

In [15]:
print(exp())

2


### Syntax Directed Interpretation of Statements

In order to get back to our example above that stores a value in a variable, recall the grammar snippet for 
the non-terminal `stmt` in Exp1,
```
stmt : PRINT exp ';'
     | STORE var exp ';'
```
The corresponding parsing function looks like this,

In [16]:
# %load -s stmt code/exp1_rinterp
def stmt():
    tok = token_stream.pointer()
    
    if tok.type == 'PRINT':
        token_stream.next() # match PRINT
        print("> {}".format(exp()))
        token_stream.next() # match ;
        return None
    
    elif tok.type == 'STORE':
        token_stream.next() # match STORE
        name = lvar() # not var()!
        val = exp()
        symbol_table[name] = val
        token_stream.next() # match ;
        return None
    
    else:
        raise SyntaxError('unexpected symbol {} while parsing'.format(tok.value))


The first thing to notice is that in Exp1 statements themselves do not compute any values and
therefore the corresponding parsing function does not return any values.
Looking at the function itself we see that in the case of a `PRINT` statement we compute the value of the expression
while parsing it and then write that value to the output.
In terms of the `STORE` statement we have to be careful with lvalues and rvalues of variables.  Computing rvalues of
variables is straight forward as we saw in the `var()` function.  It is simply a matter of looking up the corresponding value in a symbol table.  In a store the name of the variable acts like an lvalue in the sense that it serves as a handle for a location to update.  In our case, the name of the variable is the key into a dictionary
and as you see in the `stmt()` function we use that key to update the variable in the symbol table.
So the `lvar()` function returns the name of the variable whereas the `var()` function returns the value associated with a variable - lvalue *vs.* rvalue.

Here is the definition of the `lvar()` function.

In [17]:
# %load -s lvar code/exp1_rinterp
def lvar():
    tok = token_stream.pointer()
    
    if tok.type == 'NAME':
        token_stream.next()
        return tok.value # return var name
    
    else:
        raise SyntaxError('unexpected symbol {} while parsing'.format(tok.value))


### Interpreting Statements

Back to our example, we want to interpret the statement `store y + 2 x ;`.  We need to set up our input and token streams appropriately and reinitialize our symbol table,

In [18]:
input_stream = "store y + 2 x ;"
token_stream = TokenStream(lexer, input_stream)

In [19]:
symbol_table = dict()

In [20]:
stmt()

In [21]:
print(symbol_table)

{'y': 2}


The contents of the symbol table is exactly what we had expected given that the default value for the variable `x` is zero since nothing had been assigned to it.  In order to change that we preload the symbol table with a value for `x`
and then set up the input and token streams accordingly,

In [22]:
symbol_table = {'x':3}

In [23]:
input_stream = "store y + 2 x ;"
token_stream = TokenStream(lexer, input_stream)

In [24]:
stmt()

In [25]:
print(symbol_table)

{'x': 3, 'y': 5}


<!--
\index{rvalue}
\index{lvalue}
\index{symbol table}
-->

<!-- This paragraph makes no sense here...
Just as in the program from the previous chapter where we tried to find all the variable references were not variable definitions in an Exp0 program
we have to be careful with the interpretation of Exp1 programs and distinguish lvalues and rvalues.
If a variable appears as an lvalue (that is it appears as the first argument to the STORE statement) then we assign a value to it and
if a variable appears as an rvalue (that is it appears in the expression of the STORE statement) then we just look up the corresponding 
value for the variable.
Value updates and lookups are usually accomplished with the help of a symbol table. 
Exp1 is simple enough that a simple dictionary like table as a way to associate variable names with values suffices.
-->

The following video shows an animation of the syntax directed interpretation of our Exp1 program:

<!-- videos/chap02/q7/figure.mov -->

<a href="http://www.youtube.com/watch?feature=player_embedded&v=jmE_9zOfp1g" target="_blank">
<img style='border:1px solid #000000' src="movie.jpg" width="120" height="90" />
</a>

### Processing Statement Lists

Rather than preloading the symbol table with a value for `x` we can  initialize `x` through a `store` statement and we can print out the value of `y` with an Exp1 `print` statement after `y` has been updated.  Adding statment lists to our parser will allow us to do that.  Recall the grammar snippet that specifies statement lists,
```
stmt_list : stmt_list stmt
          | empty

```
The empty rule makes it difficult to convert the grammar snippet into
a recursive descent parser function.
However, we can rewrite these rules borrowing some notation from regular expressions,
```
stmt_list : stmt*
```
meaning that a statement list consists of zero or more statements.  This allows us to construct a parser function for `stmt_list`,

In [26]:
# %load -s stmt_list code/exp1_rinterp
def stmt_list():
    while not token_stream.end_of_file():
        stmt()
    return None


In order to test this new function we need to reinitialize our symbol table and set up the streams,

In [27]:
symbol_table = dict()

input_stream = \
'''
store x 3; 
store y + 2 x; 
print y;
'''

token_stream = TokenStream(lexer, input_stream)

Here we go:

In [28]:
stmt_list()

> 5


We now have a fully functioning interpreter for our Exp1 language.  The interpreter is syntax directed because the values are being computed and passed along as we parse the source program.  

### Adding a Toplevel Driver

To create a more polished implementation of the interpreter we can add a toplevel driver function,

In [29]:
# %load -s exp1_rinterp code/exp1_rinterp
def exp1_rinterp(input_stream = None):
    'driver for our recursive descent Exp1 interpreter.'
    
    global token_stream
    global symbol_table
    
    if not input_stream:
        input_stream = input("exp1 > ")
    
    token_stream = TokenStream(lexer, input_stream)
    symbol_table = dict()
    
    stmt_list()


In [30]:
exp1_rinterp("store x 1; store y 2; print + x y;")

> 3


An additional feature to consider is to read program files rather than reading programs from strings.

## An Interpreter for Exp1 using an LR(1) Parser


<!-- TODO: rename _rinterp to _rdinterp -->

Now that we have a good handle on what syntax directed interpretation entails let us implement our Exp1 interpreter using an LR(1) parser generated by Ply.

Recall our Ply grammar for Exp1.

```Python
# %load code/exp1_gram.py
from ply import yacc
from exp1_lex import tokens, lexer

def p_grammar(_):
    """
    prog : stmt_list
    
    stmt_list : stmt stmt_list
              | empty
              
    stmt : PRINT exp ';'
         | STORE var exp ';'
         
    exp : '+' exp exp
        | '-' exp exp
        | '(' exp ')'
        | var
        | num
        
    var : NAME
        
    num : NUMBER
    """
    pass

def p_empty(p):
    'empty :'
    pass

def p_error(t):
    print("Syntax error at '%s'" % t.value)

parser = yacc.yacc()
```

The first step twoards building an interpreter with this grammar is to break all the grammar rules out into individual functions so that we can embed actions with the grammar rules.
The actions are python code that are able to access individual parts of the rules.
In particular, we take advantage of the variable `p` which the parser maintains.

The variable `p` that gets passed into each of the parsing functions is an array indexed by the tokens and non-terminals
appearing in a grammar rule.  Consider the rule,
```
exp : '+' exp exp
 0     1   2   3
```
We have written the index of `p` underneath each rule component.
Here, the value of the `+` token can be accessed with `p[1]`.
The value of the first expression after the plus sign can be accessed with `p[2]` and the value of the second expression can be accessed with `p[3]`.
Now, recall that LR parsers run grammar rules backwards.
So in this case the right side of the rule, `'+' exp exp`, will be replaced by `exp` on the stack.
Just before this happens we can assign a value to the resulting `exp` as follows,
```
p[0] = p[2] + p[3]
```
That is, just before the right side of the rule is replaced with the left side we pull the values of the expressions
on the right side, use them to compute the value of the resulting expression, and then assign that value to the 
resulting expression with the index of zero, `p[0]`.  In essence we emulated here exactly the same computations we performed in our recursive descent parser.

### The Exp1 Parser

Here is our Ply grammar with the embedded actions that make extensive use of the `p` variable.

```python
# %load code/exp1_lrinterp_gram.py
from ply import yacc
from exp1_lex import tokens, lexer

symbol_table = dict()

def p_prog(_):
    "prog : stmt_list"
    pass

def p_stmt_list(_):
    """
    stmt_list : stmt stmt_list
              | empty
    """
    pass

def p_print_stmt(p):
    "stmt : PRINT exp ';'"
    print("> {}".format(p[2]))
    
def p_store_stmt(p):
    "stmt : STORE NAME exp ';'"
    symbol_table[p[2]] = p[3]

def p_plus_exp(p):
    """
    exp : '+' exp exp
    """
    p[0] = p[2] + p[3]

def p_minus_exp(p):
    """
    exp : '-' exp exp
    """
    p[0] = p[2] - p[3]

def p_paren_exp(p):
    """
    exp : '(' exp ')'
    """
    p[0] = p[2]

def p_var_exp(p):
    "exp : var"
    p[0] = p[1]
    
def p_num_exp(p):
    "exp : num"
    p[0] = p[1]

def p_var(p):
    "var : NAME"
    p[0] = symbol_table.get(p[1], 0)

def p_num(p):
    "num : NUMBER"
    p[0] = p[1]

def p_empty(p):
    "empty :"
    pass

def p_error(t):
    print("Syntax error at '%s'" % t.value)

parser = yacc.yacc(debug=False, tabmodule='exp1parsetab')
```

One of the more interesting functions to look at in this grammar is the `p_store_stmt` function where the value of the expression `p[3]` is used
to update the symbol table at key `p[2]`.
Another interesting function is the `p_arith_exp` function where we use the value of the first token on the left side of the rule, `p[1]`, to figure
out what arithmetic operation we need to perform on the expression values `p[2]` and `p[3]`.

### Testing our LR(1) Parser

We now import the parser from that grammar to test whether our interpreter actually works.

In [31]:
from exp1_lrinterp_gram import parser

Generating LALR tables


In [32]:
parser.parse(input="store x 1; print + x 10;")

> 11


Yup, that worked!

### Adding a Toplevel Driver

Just as in the case of our recursive descent parser, we can  provide a toplevel driver function to provide a more polished interface to our interpreter.

In [ ]:
# %load -s exp1_lrinterp code/exp1_lrinterp
def exp1_lrinterp(input_stream = None):
    'A driver for our LR Exp1 interpreter.'
    
    if not input_stream:
        input_stream = input("exp1 > ")
    
    parser.parse(input_stream)


In [34]:
exp1_lrinterp("store x 1; store y 2; print + x y;")

> 3


## Another Take on Syntax-Directed Processing: A Pretty Printer for Exp1

<!--
\index{syntax directed translation}
\index{translation!syntax directed}
-->

Syntax directed language processing does not only apply to interpretation.  We can also use syntax directed techniques to build
simple translators.
A pretty printer for our Exp1 language is a good example to look at.
As you might know, pretty printers are programs that read the source of a program written in some programming language
and then generate code in the same language but formatted nicely so that the program is easy to read.
This is a great example of a simple translator shown in Figure 8 Chapter 1 except in our case it is not necessary
to construct an IR because we will use syntax directed translation.
Our pretty printer accomplishes two things: 

1. It will put each statement on its own line.  

2. Expressions will be written in Lisp like syntax.  In Lisp, each operation is embedded in a pair of parentheses.  For example, to add two numbers in Lisp we write the following expression,
```
(+ 2 3)
```
This also means we  get rid of unnecessary parentheses.  For example, the expression `((+ (2) (3)))` will be rewritten as above.
 

### The Pretty Printer Parser

The following is our Exp1 grammar extended with the embedded actions that allow us to generate the pretty printed code.

```Python
# %load code/exp1_pp_gram.py
from ply import yacc
from exp1_lex import tokens, lexer

def p_prog(p):
    "prog : stmt_list"
    print(p[1])

def p_stmt_list(p):
    "stmt_list : stmt stmt_list"
    p[0] = p[1] + p[2]

def p_stmt_list_empty(p):
    "stmt_list : empty"
    p[0] = p[1]

def p_print_stmt(p):
    "stmt : PRINT exp ';'"
    p[0] = 'print' + p[2] +';\n'

def p_store_stmt(p):
    "stmt : STORE NAME exp ';'"
    p[0] = 'store ' + p[2] + p[3] +';\n'

def p_arith_exp(p):
    """
    exp : '+' exp exp
        | '-' exp exp
        | '(' exp ')'
    """
    if p[1] == '+':
        p[0] = ' (+' + p[2] + p[3] + ')'
    elif p[1] == '-':
        p[0] = ' (-' + p[2] + p[3] + ')'
    elif p[1] == '(':
        p[0] = p[2]
    else:
        raise SyntaxError("parsing weirdness in expressions: {} !".format(p[1]))

def p_var_exp(p):
    "exp : var"
    p[0] = p[1]
    
def p_num_exp(p):
    "exp : num"
    p[0] = p[1]

def p_var(p):
    "var : NAME"
    p[0] = ' ' + p[1]

def p_num(p):
    "num : NUMBER"
    p[0] = ' ' + str(p[1])

def p_empty(p):
    "empty :"
    p[0] = ''

def p_error(t):
    print("Syntax error at '%s'" % t.value)

parser = yacc.yacc()
```

The idea in this syntax directed approach to our pretty printer is that we will assemble a string that contains the 
pretty printed program while we are parsing the source program.  In order to see this the best place to start are the rules for variables and numbers,
```Python
def p_var(p):
    "var : NAME"
    p[0] = ' ' + p[1]

def p_num(p):
    "num : NUMBER"
    p[0] = ' ' + str(p[1])
```
Let's take a look at the rule for variables.  Recall that the value of `p[1]` is the value of the corresponding
token in the rule, in this case the token `NAME`.  The value, in this case, being the actual variable name as it 
appears in the source program.  The embedded action for this grammar rule assembles the string `' ' + p[1]`,
that is a space followed by the name of the variable, and assigns this to the non-terminal `var`.
The space is necessary for formatting reasons so that names and symbols in the pretty printed program are separated by
white space.

Something very similar happens in the rule for numbers.  Here we generate the string `' ' + p[1]` where `p[1]` 
has the numeric value of the token `NUMBER`.  The generated string is then assigned to the non-terminal `num`.

Now that we know what the leaves of an expression generate for pretty printed strings we will take a look
at the rules for expressions. Let's start at the expression rules for variables and numbers, 
```Python

def p_var_exp(p):
    "exp : var"
    p[0] = p[1]
    
def p_num_exp(p):
    "exp : num"
    p[0] = p[1]
```
In both cases the embedded actions just copy the string value from the non-terminals `var` and `num`, respectively,
and assign it to the non-terminal `exp` on the left side of the grammar rule.

We are now ready to look at the rules for the other expressions which are the addition and subtraction
operations and the parenthesized expressions. 
```Python
def p_arith_exp(p):
    """
    exp : '+' exp exp
        | '-' exp exp
        | '(' exp ')'
    """
    if p[1] == '+':
        p[0] = ' (+' + p[2] + p[3] + ')'
    elif p[1] == '-':
        p[0] = ' (-' + p[2] + p[3] + ')'
    elif p[1] == '(':
        p[0] = p[2]
    else:
        raise SyntaxError("parsing weirdness in expressions: {} !".format(p[1]))
```
The embedded action for the addition operation computes a string that contains the parenthesized addition operator
together with its arguments from the two expressions, `p[2]` and `p[3]`.  If the souce program contained the 
expression `+ x 1` then the embedded action for the addition operator will compute the string `' (+ x 1)'`.
Similarly for the subtraction operator.

Parentheses from the source program are ignored.  The rule for parenthesized expressions just copies
the string from the expression on the right of the rule to the left of the rule ignoring the parentheses.

The embedded actions for print and store statements should now be fairly obvious.  They each assemble nicely formatted
strings for the respective commands.  Notice the `\n` character at the end of the strings which means that each 
print and store command will be on its own line as required.

Finally, the embedded actions for statement lists generate one long string for the pretty printed program.
The embedded action for the non-terminal `prog`, our start symbol, print the string containing the pretty printed
program to the terminal.


### Testing the Pretty Printer

In [35]:
from exp1_pp_gram import parser

Generating LALR tables


In [36]:
parser.parse("store x 1; store y 2; print + x y;")

store x 1;
store y 2;
print (+ x y);



The pretty printer works as we wanted it to.  Each statement is on its own line and expressions written in Lisp syntax.

### Putting all together

In order to pull this all together into a nice tidy program we can add a toplevel driver as we did for the interpreter.

In [37]:
# %load code/exp1_pp.py
from exp1_pp_gram import parser

def exp1_pp(input_stream = None):
    'A driver for our Exp1 pretty printer.'
    
    if not input_stream:
        input_stream = input("exp1 > ")
    
    parser.parse(input_stream)



In [38]:
exp1_pp("store x 1; store y 2; print + x y;")

store x 1;
store y 2;
print (+ x y);



# Summary

For a certain class of programming languages we can construct syntax directed processors.  That is, we can construct processors that can execute some semantic action when relevant syntactic structures have been recognized.  We show that Exp1 can be processed in a syntax directed manner by first constructing an interpreter based on a recursive descent parser and then by constructing an interpreter based on an LR(1) parser generated by Ply.
Syntax directed techniques can also be applied to translators.  We showed this by constructing a pretty printer for Exp1.

# Bibliographic Notes

The earliest reference to syntax directed translation we found is (Irons, 1961). A more modern take on syntax directed techniques can be found in (Aho *et al*, 1986).


Irons, E. T. (1961). *A syntax directed compiler for ALGOL 60*. Communications of the ACM, 4(1), 51-55.

Aho, A. V., Sethi, R., & Ullman, J. D. (1986). *Compilers, Principles, Techniques* . Boston: Addison-Wesley.



# Exercises

<!-- TODO: More exercises -->

1. (project) Use the code for the Exp1 interpreter from above and extend the language with multiplication and integer division.  Demonstrate that your interpreter works by running it on some telling examples.
 
2. Write some additional programs in Exp1 and run an interpreter on them.
```Python
from exp1_lrinterp_gram import parser
parser.parse("store x 1; store y 2; print + x y;")
```

3. (project) Design a language and construct a syntax directed interpreter for it.